# Initialize the dataframes

In [202]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks

import pandas as pd
import random

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks


# Year 1 Constraints

Constraints:
*   Inpt cannot occur two months in a row for a student
*   Each month two students must have Inpt
* Inpt cannot be in a vacation month
* Peds cannot be in a vacation month
* Each student must take OBGYN twice
* Each student must take Peds twice
* Each student must take NICU, Emergency Med, Psych, and Surg one time

Classes: Inpt, OBGYN, Peds, NICU, Emergency Med, Psych, Surg


Missing:
*   Each month two students must have Inpt
* Other classes besides Inpt can only have one person in it per month
* Inpt cannot occur twice in a row

In [203]:
def assign_numbers(people, vacation_months):
    assigned_numbers = {}
    for person in people:
        assigned_numbers[person] = {}
        for month in range(1, 13):
            assigned_numbers[person][month] = 0

        # Assign 4 instances of number 1, 2 instances of number 2, 2 instances of number 3,
        # and 1 instance of numbers 4, 5, 6, and 7 for each person
        assigned_months = set()
        while len(assigned_months) < 4:
            month = random.randint(1, 12)
            if assigned_numbers[person][month] == 0 and month not in vacation_months[person]:
                assigned_numbers[person][month] = 1
                assigned_months.add(month)

        assigned_twos = set()
        while len(assigned_twos) < 2:
            month = random.randint(1, 12)
            if assigned_numbers[person][month] == 0:
                assigned_numbers[person][month] = 2
                assigned_twos.add(month)

        assigned_threes = set()
        while len(assigned_threes) < 2:
            month = random.randint(1, 12)
            if assigned_numbers[person][month] == 0 and month not in vacation_months[person]:
                assigned_numbers[person][month] = 3
                assigned_threes.add(month)

        assigned_others = set()
        for number in range(4, 8):
            month = random.randint(1, 12)
            while assigned_numbers[person][month] != 0 or number == 1 and assigned_numbers[person].count(1) > 1:
                month = random.randint(1, 12)
            assigned_numbers[person][month] = number
            assigned_others.add(month)

        # Assign the remaining months randomly, with the constraint that only the number 1 can repeat in a month
        remaining_months = set(range(1, 13)) - assigned_months - assigned_twos - assigned_threes - assigned_others
        prev_num = 0
        for month in remaining_months:
            available_numbers = set(range(1, 8)) - set([assigned_numbers[person][m] for m in assigned_numbers[person] if m != month])
            if 1 in available_numbers and assigned_numbers[person][month] != 1:
                assigned_numbers[person][month] = 1
            else:
                if prev_num == 1:
                    available_numbers = available_numbers - set([1])
                assigned_numbers[person][month] = random.choice(list(available_numbers))
            prev_num = assigned_numbers[person][month]

    df = pd.DataFrame(assigned_numbers)
    return df

In [204]:
# Example usage
people = ['Joseph Caravella', 'Jordan Coffey', 'Marcus Davis', 'Jackson Weaver', 'J.J. Westmoreland', 'Hannah "Day" Willey']
vacation_months = {'Joseph Caravella': [1,2,3,4],
                   'Jordan Coffey': [5,6,7,8],
                   'Marcus Davis': [9,10,11,12],
                   'Jackson Weaver': [2,4,6,8],
                   'J.J. Westmoreland': [1,3,5,7],
                   'Hannah "Day" Willey': [1,2,3,4]}
numbers = list(range(1, 8))
assignments = assign_numbers(people,vacation_months)

In [205]:
num_to_rotation = {
    1: 'Inpt',
    2: 'OBGYN',
    3: 'Peds',
    4: 'NICU',
    5: 'Emergency Med',
    6: 'Psych',
    7: 'Surg'
}
# Replace numbers with letters using the map() method
assignments.index.name = 'Month'
for ind in assignments:
  assignments[ind] = assignments[ind].map(num_to_rotation)
df1=assignments
df1

,Joseph Caravella,Jordan Coffey,Marcus Davis,Jackson Weaver,J.J. Westmoreland,"Hannah ""Day"" Willey"
Month,,,,,,
1,NICU,OBGYN,Surg,NICU,NICU,NICU
2,OBGYN,Inpt,Psych,OBGYN,Inpt,Psych
3,Emergency Med,Peds,Peds,Inpt,OBGYN,Surg
4,Surg,Inpt,Inpt,Emergency Med,Peds,Emergency Med
5,Inpt,NICU,Inpt,Inpt,Emergency Med,Inpt
6,Psych,OBGYN,Peds,Surg,Inpt,Inpt
7,Peds,Surg,Inpt,Inpt,Surg,Inpt
8,OBGYN,Emergency Med,Inpt,OBGYN,OBGYN,Peds
9,Peds,Inpt,OBGYN,Peds,Peds,Peds


# Year 2 Constraints

Constraints:
* Each student needs Inpt 4 times
* Inpt needs two people in it each month  
* Inpt cannot be in the same month as a vacation
* EM UMC cannot be in the same month as a vacation
* Everything else is needed only 1 time (Gyn,Cards,Crit Care,Amb Peds, EM UMC, Practice Mngmt, Peds EM, Ortho)
* All classes besides INPT will have 1 student per month max

Classes: Gyn, Cards, Crit Care, Amb Peds, EM UMC, Practice Mngmt, Peds EM, Ortho, Inpt

Missing:
*   Each month two students must have Inpt
* Other classes besides Inpt can only have one person in it per month
* Inpt cannot occur twice in a row

In [206]:
def assign_numbers(people, vacation_months):
    assigned_numbers = {}
    for person in people:
        assigned_numbers[person] = {}
        for month in range(1, 13):
            assigned_numbers[person][month] = 0

        # Assign 4 instances of number 1, 2 instances of number 2, 2 instances of number 3,
        # and 1 instance of numbers 4, 5, 6, and 7 for each person
        assigned_months = set()
        while len(assigned_months) < 4:
            month = random.randint(1, 12)
            if assigned_numbers[person][month] == 0 and month not in vacation_months[person]:
                assigned_numbers[person][month] = 1
                assigned_months.add(month)

        assigned_twos = set()
        while len(assigned_twos) < 1:
            month = random.randint(1, 12)
            if assigned_numbers[person][month] == 0 and month not in vacation_months[person]:
                assigned_numbers[person][month] = 2
                assigned_twos.add(month)

        assigned_others = set()
        for number in range(2, 9):
            month = random.randint(1, 12)
            while assigned_numbers[person][month] != 0 or number == 1 and assigned_numbers[person].count(1) > 1:
                month = random.randint(1, 12)
            assigned_numbers[person][month] = number
            assigned_others.add(month)

        # Assign the remaining months randomly, with the constraint that only the number 1 can repeat in a month
        remaining_months = set(range(1, 13)) - assigned_months - assigned_twos - assigned_others
        prev_num = 0
        for month in remaining_months:
            available_numbers = set(range(1, 9)) - set([assigned_numbers[person][m] for m in assigned_numbers[person] if m != month])
            if 1 in available_numbers and assigned_numbers[person][month] != 1:
                assigned_numbers[person][month] = 1
            else:
                if prev_num == 1:
                    available_numbers = available_numbers - set([1])
                assigned_numbers[person][month] = random.choice(list(available_numbers))
            prev_num = assigned_numbers[person][month]

    df = pd.DataFrame(assigned_numbers)
    return df

In [207]:
people = ['Cody Bushyhead', 'Prabh Khunkhun', 'Jay Napolitano', 'Andrew Oracz', 'Natasha Rezvani', 'Steven Vo']
vacation_months = {'Cody Bushyhead': [9, 12, 3, 6],
                   'Prabh Khunkhun': [9, 12, 4,6],
                   'Jay Napolitano': [11, 3, 5,6],
                   'Andrew Oracz': [10, 2, 4,5,6],
                   'Natasha Rezvani': [8, 12, 2,4],
                   'Steven Vo': [12, 1, 3,5]}
numbers = list(range(1, 9))
assignments = assign_numbers(people,vacation_months)

In [208]:
num_to_rotation = {
    1: 'Inpt',
    2: 'Ortho',
    3: 'Peds',
    4: 'Practice Mngmt',
    5: 'EM UMC',
    6: 'Amb Peds',
    7: 'Crit Care',
    8: 'Gyn'
}
# Replace numbers with letters using the map() method
assignments.index.name = 'Month'
for ind in assignments:
  assignments[ind] = assignments[ind].map(num_to_rotation)
df2=assignments
df2

,Cody Bushyhead,Prabh Khunkhun,Jay Napolitano,Andrew Oracz,Natasha Rezvani,Steven Vo
Month,,,,,,
1,Inpt,Inpt,Peds,Inpt,Inpt,Gyn
2,Inpt,EM UMC,Amb Peds,EM UMC,Amb Peds,Inpt
3,Ortho,Inpt,EM UMC,Ortho,EM UMC,Ortho
4,Inpt,Gyn,Inpt,Amb Peds,Practice Mngmt,Inpt
5,Ortho,Peds,Crit Care,Peds,Crit Care,Crit Care
6,Crit Care,Crit Care,Gyn,Crit Care,Ortho,Inpt
7,Peds,Ortho,Ortho,Inpt,Inpt,Ortho
8,Amb Peds,Inpt,Ortho,Inpt,Peds,Practice Mngmt
9,EM UMC,Practice Mngmt,Inpt,Inpt,Inpt,Amb Peds


# Year 3 Constraints

Constraints:
* Each student needs Inpt 2 times
* Each student needs Elective 4 times
* Elective needs two people in it each month  
* Inpt cannot be in the same month as a vacation
* ENT/Opthal and Uro/Neuro cannot be in a vacation month
* Everything else is needed only 1 time (ENT/Ophthal, MSK, Amb Peds, Uro/Neuro, Comm Med, Derm)
* All classes besides Elective will have 1 student per month max

Classes: Inpt, Elective, ENT/Ophthal, MSK, Amb Peds, Uro/Neuro, Comm Med, Derm


Missing:
* All classes besides Elective will have 1 student per month max
* Elective needs two people in it each month
* Inpt cannot occur two months in a row

In [209]:
def assign_numbers(people, vacation_months):
    assigned_numbers = {}
    for person in people:
        assigned_numbers[person] = {}
        for month in range(1, 13):
            assigned_numbers[person][month] = 0

        # Assign 4 instances of number 1, 2 instances of number 2, 2 instances of number 3,
        # and 1 instance of numbers 4, 5, 6, and 7 for each person
        assigned_months = set()
        while len(assigned_months) < 2:
            month = random.randint(1, 12)
            if assigned_numbers[person][month] == 0 and month not in vacation_months[person]:
                assigned_numbers[person][month] = 1
                assigned_months.add(month)

        assigned_twos = set()
        while len(assigned_twos) < 4:
            month = random.randint(1, 12)
            if assigned_numbers[person][month] == 0:
                assigned_numbers[person][month] = 2
                assigned_twos.add(month)

        assigned_threes = set()
        while len(assigned_threes) < 1:
            month = random.randint(1, 12)
            if assigned_numbers[person][month] == 0 and month not in vacation_months[person]:
                assigned_numbers[person][month] = 3
                assigned_threes.add(month)
        
        assigned_fours = set()
        while len(assigned_threes) < 1:
            month = random.randint(1, 12)
            if assigned_numbers[person][month] == 0 and month not in vacation_months[person]:
                assigned_numbers[person][month] = 4
                assigned_threes.add(month)

        assigned_others = set()
        for number in range(5, 9):
            month = random.randint(1, 12)
            while assigned_numbers[person][month] != 0 or number == 1 and assigned_numbers[person].count(1) > 1:
                month = random.randint(1, 12)
            assigned_numbers[person][month] = number
            assigned_others.add(month)

        # Assign the remaining months randomly, with the constraint that only the number 1 can repeat in a month
        remaining_months = set(range(1, 13)) - assigned_months - assigned_twos - assigned_threes - assigned_others
        prev_num = 0
        for month in remaining_months:
            available_numbers = set(range(1, 9)) - set([assigned_numbers[person][m] for m in assigned_numbers[person] if m != month])
            if 1 in available_numbers and assigned_numbers[person][month] != 1:
                assigned_numbers[person][month] = 1
            else:
                if prev_num == 1:
                    available_numbers = available_numbers - set([1])
                assigned_numbers[person][month] = random.choice(list(available_numbers))
            prev_num = assigned_numbers[person][month]

    df = pd.DataFrame(assigned_numbers)
    return df

In [210]:
people = ['Bryce Chapman', 'Charlie Dean', 'Read Founds', 'Marshall Frieden', 'Shonte Joseph', 'Chris Van Hise']
vacation_months = {'Bryce Chapman': [11, 3, 5,6],
                   'Charlie Dean': [8, 10, 1, 2],
                   'Read Founds': [11, 12, 2,4],
                   'Marshall Frieden': [8, 12, 2,3],
                   'Shonte Joseph': [7, 12, 2,4],
                   'Chris Van Hise': [8, 11, 2,5]}
numbers = list(range(1, 9))
assignments = assign_numbers(people,vacation_months)

In [211]:
num_to_rotation = {
    1: 'Inpt',
    2: 'Elective',
    3: 'ENT/Opthal',
    4: 'Uro/Neuro',
    5: 'MSK',
    6: 'Amb Peds',
    7: 'Comm Med',
    8: 'Derm'
}
# Replace numbers with letters using the map() method
assignments.index.name = 'Month'
for ind in assignments:
  assignments[ind] = assignments[ind].map(num_to_rotation)
df3=assignments
df3

,Bryce Chapman,Charlie Dean,Read Founds,Marshall Frieden,Shonte Joseph,Chris Van Hise
Month,,,,,,
1,Elective,Derm,MSK,Uro/Neuro,Elective,Inpt
2,Comm Med,Elective,Elective,Elective,Elective,Amb Peds
3,Uro/Neuro,Inpt,Comm Med,Comm Med,Derm,Inpt
4,Inpt,ENT/Opthal,Elective,Elective,Amb Peds,Derm
5,Elective,Inpt,Elective,ENT/Opthal,Comm Med,Elective
6,Derm,MSK,Elective,Inpt,Inpt,Comm Med
7,Amb Peds,Elective,Derm,MSK,Elective,Elective
8,Elective,Elective,ENT/Opthal,Derm,Elective,Uro/Neuro
9,MSK,Elective,Inpt,Elective,Inpt,ENT/Opthal
